# Quick Start: SAR + AIS Data Download

A simple example showing how to search for Sentinel-1 SAR data and download corresponding AIS vessel tracking data for the same area and time.

In [ ]:
# Import phidown modules
from phidown import CopernicusDataSearcher
from phidown.ais import download_ais_data
import pandas as pd

In [ ]:
# Define area of interest (Mediterranean Sea near shipping lanes)
aoi_wkt = """POLYGON((
    10.0 40.0,
    15.0 40.0,
    15.0 45.0,
    10.0 45.0,
    10.0 40.0
))"""

# Set date for analysis
target_date = "2024-07-15"

print(f"Searching for data on {target_date}")
print(f"Area: Mediterranean Sea (10-15°E, 40-45°N)")

In [ ]:
# Search for Sentinel-1 SAR data
searcher = CopernicusDataSearcher()

sar_results = searcher.search(
    collection_name='SENTINEL-1',
    aoi_wkt=aoi_wkt,
    start_date=target_date,
    end_date=target_date,
    product_type='GRD'  # Ground Range Detected
)

print(f"Found {len(sar_results)} SAR products")

if len(sar_results) > 0:
    # Show first few results
    display_cols = ['Name', 'ContentDate', 'Instrument', 'ProductType']
    available_cols = [col for col in display_cols if col in sar_results.columns]
    print("\nSAR Products:")
    print(sar_results[available_cols].head(3))

In [ ]:
# Download AIS data for the same area and time
try:
    ais_data = download_ais_data(
        start_date=target_date,
        start_time="08:00:00",  # Morning period
        end_time="16:00:00",   # Afternoon period
        aoi_wkt=aoi_wkt
    )
    
    print(f"Downloaded {len(ais_data)} AIS records")
    
    if len(ais_data) > 0:
        print(f"\nAIS Data Overview:")
        print(f"  Unique vessels: {ais_data['mmsi'].nunique()}")
        print(f"  Named vessels: {len(ais_data[ais_data['name'] != ''])}")
        print(f"  Data columns: {list(ais_data.columns)}")
        
        # Show sample data
        print(f"\nSample AIS records:")
        print(ais_data[['name', 'lat', 'lon', 'timestamp', 'mmsi']].head())
    
except Exception as e:
    print(f"AIS download failed: {e}")
    print("Make sure you have installed: pip install phidown[ais]")

In [ ]:
# Simple analysis: vessel positions vs SAR coverage
if len(sar_results) > 0 and len(ais_data) > 0:
    print("Data Integration Summary:")
    print(f"  SAR products available: {len(sar_results)}")
    print(f"  AIS vessel positions: {len(ais_data)}")
    print(f"  Potential vessel-SAR matches: Ready for analysis")
    
    # Basic statistics
    print(f"\nSpatial Coverage:")
    print(f"  Latitude range: {ais_data['lat'].min():.2f} to {ais_data['lat'].max():.2f}")
    print(f"  Longitude range: {ais_data['lon'].min():.2f} to {ais_data['lon'].max():.2f}")
    
    # Save combined results
    ais_data.to_csv(f'ais_data_{target_date}.csv', index=False)
    sar_results.to_csv(f'sar_results_{target_date}.csv', index=False)
    
    print(f"\nData saved to:")
    print(f"  - ais_data_{target_date}.csv")
    print(f"  - sar_results_{target_date}.csv")
else:
    print("No data available for integration analysis.")

## Next Steps

Now you have:
1. **SAR search results** - showing available satellite imagery
2. **AIS vessel data** - showing ship positions and movements

You can:
- Download the actual SAR imagery using phidown's download functions
- Analyze vessel detection in SAR images
- Compare AIS positions with SAR detections
- Study maritime traffic patterns

See the detailed notebook `4_sar_and_ais_integration.ipynb` for advanced analysis examples.